In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
station_df = pd.read_csv('./station.csv')
station_df.head(2)

,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date
0,BT-01,3rd Ave & Broad St,47.618418,-122.350964,10/13/2014,18,NaN,18,NaN
1,BT-03,2nd Ave & Vine St,47.615829,-122.348564,10/13/2014,16,NaN,16,NaN


In [3]:
trip_df = pd.read_csv('./trip.csv', error_bad_lines=False)
trip_df.head(2)

b'Skipping line 50794: expected 12 fields, saw 20\n'


,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear
0,431,10/13/2014 10:31,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1960.0
1,432,10/13/2014 10:32,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1970.0


In [4]:
weather_df = pd.read_csv('./weather.csv')
weather_df.head(2)

,Date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,...,Mean_Sea_Level_Pressure_In,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events
0,10/13/2014,71,62.0,54,55,51,46,87,68,46,...,29.79,29.65,10,10,4,13,4,21,0.00,Rain
1,10/14/2014,63,59.0,55,52,51,50,88,78,63,...,29.75,29.54,10,9,3,10,5,17,0.11,Rain


# What is the average trip duration for a borrowed bicycle?

In [5]:
trip_mins = trip_df.tripduration.mean() // 60
print(f'Average trip duration was {trip_mins} minutes')

Average trip duration was 19.0 minutes


# What's the most common age of a bicycle sharer?

In [6]:
trip_df['age'] = dt.now().year - trip_df['birthyear']
pd.DataFrame(trip_df['age'].mode(), columns=['Most Common Age'])

,Most Common Age
0,31.0


# Given all the weather data, find the average precipitation per month, and the median precipitation.

In [7]:
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
weather_df['Month'] = weather_df['Date'].dt.month
weather_df.head(1)

,Date,Max_Temperature_F,Mean_Temperature_F,Min_TemperatureF,Max_Dew_Point_F,MeanDew_Point_F,Min_Dewpoint_F,Max_Humidity,Mean_Humidity,Min_Humidity,...,Min_Sea_Level_Pressure_In,Max_Visibility_Miles,Mean_Visibility_Miles,Min_Visibility_Miles,Max_Wind_Speed_MPH,Mean_Wind_Speed_MPH,Max_Gust_Speed_MPH,Precipitation_In,Events,Month
0,2014-10-13,71,62.0,54,55,51,46,87,68,46,...,29.65,10,10,4,13,4,21,0.0,Rain,10


In [8]:
avg_precip = pd.DataFrame(weather_df['Precipitation_In']).groupby(weather_df.Month).mean()
avg_precip

,Precipitation_In
Month,
1,0.143548
2,0.168421
3,0.156935
4,0.051333
5,0.012419
6,0.030500
7,0.012097
8,0.018226
9,0.041000


In [9]:
weather_df.Precipitation_In.mean() / 12

0.00875544267053701

# When a bike station is modified, is it more or less likely that it'll lose bikes or gain bikes? How do you know?

In [11]:
station_df['modification_date'] = station_df['modification_date'].notnull()


58

In [16]:
modified = station_df[['station_id', 'modification_date', 'install_dockcount', 'current_dockcount']]
filtered = modified.query('modification_date')

In [25]:
filtered[filtered['current_dockcount'] > 0]

,station_id,modification_date,install_dockcount,current_dockcount
7,CBD-06,True,20,18
12,CH-02,True,18,20
17,CH-08,True,28,26
22,DPD-03,True,30,24
23,EL-01,True,12,16
31,SLU-01,True,18,20
35,SLU-15,True,12,20
37,SLU-17,True,18,16
39,SLU-19,True,12,16
47,UW-02,True,12,14
